In [1]:
import cx_Oracle
import csv

In [2]:
connection = cx_Oracle.connect("SYSTEM", "1111", "xe")

In [3]:
cursor_person = connection.cursor()
cursor_person.execute("""
SELECT
    TRIM(person_id) as person_id,
    TRIM(person_name) as person_name
FROM
    Person""")

for person_id, person_name in cursor_person:
    with open("person_" + person_id + ".csv", "w", newline="") as file:
        writer = csv.writer(file) 
        
        writer.writerow(["ID", person_id])
        writer.writerow(["Name", person_name])

        cursor_songs = connection.cursor()
        query = """
                    SELECT
                        Song.SONG_NAME,
                        Song.SONG_ID,
                        Dance.DANCE_DATE
                    FROM
                        Dance 
                            JOIN Person ON Dance.PERSON_ID = Person.PERSON_ID
                            JOIN Song ON Song.SONG_ID = Dance.SONG_ID
                    WHERE 
                        Person.PERSON_ID = :pid
                """
        
        cursor_songs.execute(query, pid=person_id)
        writer.writerow([])
        writer.writerow(["Song name", "Song id", "Dance date"])
        
        for song_row in cursor_songs:
            writer.writerow(song_row)
            
cursor_person.close()